In [22]:
populations_sumstats_file_path = "data/long_populations.sumstats.tsv"
snp_cline_parameters_file_path = "data/long_p8_snp_cline_parameters.filtered.tsv"

out_snp_cline_parameters_file_path = "long.p8.out.snp_cline_parameters_with_parental_allele_freq_diff.tsv"

# Parental Populations of interest - the two non-hybrid parental populations. 
# NOTE: Script only supports two parental populations.
parental_populations = ('cg_100', 'ss_020')

In [23]:
# TODO use named tuple to make it easier to pull things out
# from collections import namedtuple
# RelevantSumstatsSubset = namedtuple('RelevantSumstatsSubset', 'snp_id p')

In [24]:
filtered_population_sumstats = []
with open(populations_sumstats_file_path, "r") as populations_sumstats_file:
    lines=populations_sumstats_file.readlines()
    
    until= len(lines)
    for i in range(0,until):
        line = lines[i]

        # Get rid of headers -  If comment line, skip
        if line.startswith('#'):
            continue

        [locus_id, chr, bp, col, pop_id, p_nuc, q_nuc, n, p, *_ ] = line.split()
        if pop_id in parental_populations:
            # Create the snp_id "<Locus ID>_<Col>"
            filtered_population_sumstats.append((f"{locus_id}_{col}", p))

print(len(filtered_population_sumstats))

print(filtered_population_sumstats[-2])
print(filtered_population_sumstats[-1])


144296
('426521_76', '0.9787234')
('426521_76', '1')


In [25]:
snp_parental_diffs = []
for i in range(0, len(filtered_population_sumstats), 2):
    p1 = filtered_population_sumstats[i]
    p2 = filtered_population_sumstats[i+1]
    diff = abs(float(p1[-1]) - float(p2[-1]))
    snp_parental_diffs.append((p1[0], diff))

print(len(snp_parental_diffs))
print(snp_parental_diffs[-1])

72148
('426521_76', 0.02127659999999998)


Now that we have snp ids and diffs
Put that in the cline parameters file
What's the default value: empty between tabs? NO, every row in cline parameters should have
Sumstats is superset of cline parameters file.
However, note that if you can't find the row in cline parameters, skip
It's not performant because we need to scan through the clien parameters per item to find and append, however we can pull the entire cline parameters file into memory at the very least so avoid multiple file I/O operations. Load file once, do all the non-performant scans+appends in-memory, and then dump to file at the end.

In [26]:
with open(snp_cline_parameters_file_path, "r") as snp_cline_parameters_file:
    snp_cline_parameters_lines = snp_cline_parameters_file.readlines()

In [27]:
# SLOW ALGORITHM: Now for the super non-performant brute-force part.
# We have to look through the snp_cline_parameters_lines each time
# since there are no guarantees of sorting and we have to preserve order.

# for i in range(0, len(snp_parental_diffs)):
#     snp_parental_diff = snp_parental_diffs[i]

#     for j in range(0, len(snp_cline_parameters_lines)):
#         snp_cline_parameters_line = snp_cline_parameters_lines[j]
        
#         [_, snp_id, *_ ] = snp_cline_parameters_line.split()

#         if snp_parental_diff[0] == snp_id:
#             snp_cline_parameters_line = snp_cline_parameters_line.rstrip() + f"\t{snp_parental_diff[1]}\n"
#             snp_cline_parameters_lines[j] = snp_cline_parameters_line

In [28]:
# FASTER ALGORITHM (which is still slow)
# Since the snp_clines params are a subset of the sumstats
# and thus are considerably smaller, 
# iterate through the snp_cline_parameters, scanning through the sumstats
# This lower-bounds the algorithm to the size of cline params which is smaller.

print(f"Processing snp_cline_parameters_lines {len(snp_cline_parameters_lines)} x snp_parental_diffs {len(snp_parental_diffs)}", )
for i in range(1, len(snp_cline_parameters_lines)):
    if i % 1000 == 0:
        print("PROGRESS snp_cline_parameters_lines", f"i={i}/{len(snp_cline_parameters_lines)}")

    snp_cline_parameters_line = snp_cline_parameters_lines[i]
    [_, snp_id, *_ ] = snp_cline_parameters_line.split()

    found = False
    for j in range(0, len(snp_parental_diffs)):
        snp_parental_diff = snp_parental_diffs[j]        

        if snp_parental_diff[0] == snp_id:
            snp_cline_parameters_line = snp_cline_parameters_line.rstrip() + f"\t{snp_parental_diff[1]}\n"
            snp_cline_parameters_lines[i] = snp_cline_parameters_line
            found = True
            continue
    
    if not found:
        # This should technically never happen... if it does, there's a problem.
        print("Found an NA", f"i={i} j={j}", snp_cline_parameters_line)
        snp_cline_parameters_line = snp_cline_parameters_line.rstrip() + f"\tNA\n"
        snp_cline_parameters_lines[i] = snp_cline_parameters_line


Processing snp_cline_parameters_lines 47792 x snp_parental_diffs 72148
PROGRESS snp_cline_parameters_lines i=1000/47792
PROGRESS snp_cline_parameters_lines i=2000/47792
PROGRESS snp_cline_parameters_lines i=3000/47792
PROGRESS snp_cline_parameters_lines i=4000/47792
PROGRESS snp_cline_parameters_lines i=5000/47792
PROGRESS snp_cline_parameters_lines i=6000/47792
PROGRESS snp_cline_parameters_lines i=7000/47792
PROGRESS snp_cline_parameters_lines i=8000/47792
PROGRESS snp_cline_parameters_lines i=9000/47792
PROGRESS snp_cline_parameters_lines i=10000/47792
PROGRESS snp_cline_parameters_lines i=11000/47792
PROGRESS snp_cline_parameters_lines i=12000/47792
PROGRESS snp_cline_parameters_lines i=13000/47792
PROGRESS snp_cline_parameters_lines i=14000/47792
PROGRESS snp_cline_parameters_lines i=15000/47792
PROGRESS snp_cline_parameters_lines i=16000/47792
PROGRESS snp_cline_parameters_lines i=17000/47792
PROGRESS snp_cline_parameters_lines i=18000/47792
PROGRESS snp_cline_parameters_lines i=

Next you need to make the snp_id from the sumstats file match what is in the cline parameters file. Each cline in the cline parameters file has a unique snp in the form of `<Locus ID>_<Col>` which was originally made from the sumstats file. Column 1 in the sumstat file is Locus ID and column 4 is Col.

Note that there are a bunch of snps in the sumstats file that will NOT be in the cline parameters file. That’s ok, but make sure the python script moves on if it doesn’t find a snp ID in the cline parameters file.


In [29]:

# One last thing: Add a column header on the first line
snp_cline_parameters_lines[0] = snp_cline_parameters_lines[0].rstrip() + f"\tparent_allele_freq_diff\n"

# Output the cline parameters file
with open(out_snp_cline_parameters_file_path, "w") as outfile:
    outfile.writelines(snp_cline_parameters_lines)